In [1]:
# some_file.py
import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, '../src')

In [2]:
import importlib as imt 
import etl
import utils
import dataset
import clustering_kproto
imt.reload(utils)
imt.reload(etl)
imt.reload(dataset)
imt.reload(clustering_kproto)

<module 'clustering_kproto' from '../src/clustering_kproto.py'>

In [3]:
from clustering_kproto import plot_costs
from clustering_kproto import Cluster_Regression
from clustering_kproto import test_cluster_prediction
from clustering_kproto import predict_one

In [4]:
import pandas as pd
import numpy as np

### prepare data

In [5]:
data = dataset.Data_2019()
data.load()
data.clean()

/home/jaejun/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [6]:
data.df.groupby(by="new_flag_name")["property_code"].count()

new_flag_name
Crowne Plaza Hotels and Resorts     25
Doubletree by Hilton                15
Embassy Suites                      18
Hampton Inn                         42
Hampton Inn and Suites              36
Hilton                              10
Hilton Garden Inn                  152
Holiday                              8
Home2                               23
Homewood Suites                     38
Others                              23
Tru by Hilton                       76
Name: property_code, dtype: int64

In [13]:
# clustering by brand
# data.df['cluster'] = data.df['new_flag_name'].astype('category')
# data.df['cluster'].cat.codes

In [8]:
#data.df.info()

### prepare features

In [7]:
features = ['num_of_rooms', 'new_flag_name', 'region', 'location_type']
X = data.df[features].to_numpy()
y = data.df["revenue"].to_numpy()

### prepare test and train set

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
#shuffle
X, y = shuffle(X, y, random_state = 77)

#split for holdout data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

### scale down numerical feature, num_of_rooms

In [9]:
X_norm = X.copy()

In [10]:
#linear norm
# X_train_norm = X_train.copy()
# X_train_norm[:,0] = X_train_norm[:,0]/np.linalg.norm(X_train_norm[:,0])
# X_test_norm = X_test.copy()
# X_test_norm[:,0] = X_test_norm[:,0]/np.linalg.norm(X_test_norm[:,0])

norm_X = np.linalg.norm(X_norm[:,0])
X_norm[:,0] = X_norm[:,0]/norm_X

In [61]:
#use z score
mu = np.mean(X_norm[:,0])
sigma = np.std(X_norm[:,0])
X_norm[:,0] = (X_norm[:,0] - mu)/sigma
np.mean(X_norm[:,0])

-9.887179296554506e-17

In [26]:
#min/max
x_max =np.max(X_norm[:,0])
x_min =np.min(X_norm[:,0])
X_norm[:,0] = (X_norm[:,0] - x_min) / (x_max - x_min)

In [19]:
#check distribution of numofrooms
import matplotlib.pyplot as plt
np.min(X_norm[:,0])

0.011027941431392805

In [21]:
X_norm[:,0] = X_norm[:,0] * 40

    Parameters
    -----------
    n_clusters : int, optional, default: 8
        The number of clusters to form as well as the number of
        centroids to generate.
    max_iter : int, default: 100
        Maximum number of iterations of the k-modes algorithm for a
        single run.
    num_dissim : func, default: euclidian_dissim
        Dissimilarity function used by the algorithm for numerical variables.
        Defaults to the Euclidian dissimilarity function.
    cat_dissim : func, default: matching_dissim
        Dissimilarity function used by the kmodes algorithm for categorical variables.
        Defaults to the matching dissimilarity function.
    n_init : int, default: 10
        Number of time the k-modes algorithm will be run with different
        centroid seeds. The final results will be the best output of
        n_init consecutive runs in terms of cost.
    init : {'Huang', 'Cao', 'random' or a list of ndarrays}, default: 'Cao'
        Method for initialization:
        'Huang': Method in Huang [1997, 1998]
        'Cao': Method in Cao et al. [2009]
        'random': choose 'n_clusters' observations (rows) at random from
        data for the initial centroids.
        If a list of ndarrays is passed, it should be of length 2, with
        shapes (n_clusters, n_features) for numerical and categorical
        data respectively. These are the initial centroids.
    gamma : float, default: None
        Weighing factor that determines relative importance of numerical vs.
        categorical attributes (see discussion in Huang [1997]). By default,
        automatically calculated from data.
    verbose : integer, optional
        Verbosity mode.
    random_state : int, RandomState instance or None, optional, default: None
        If int, random_state is the seed used by the random number generator;
        If RandomState instance, random_state is the random number generator;
        If None, the random number generator is the RandomState instance used
        by `np.random`.
    n_jobs : int, default: 1
        The number of jobs to use for the computation. This works by computing
        each of the n_init runs in parallel.
        If -1 all CPUs are used. If 1 is given, no parallel computing code is
        used at all, which is useful for debugging. For n_jobs below -1,
        (n_cpus + 1 + n_jobs) are used. Thus for n_jobs = -2, all CPUs but one
        are used.
        
        matching_dissim
        jaccard_dissim_binary
        jaccard_dissim_label
        euclidean_dissim
        ng_dissim

### clustering

In [22]:
from kmodes.kprototypes import KPrototypes
from kmodes.util.dissim import jaccard_dissim_label
from kmodes.util.dissim import matching_dissim
from kmodes.util.dissim import euclidean_dissim

kproto = KPrototypes(n_clusters = 13, init = 'Cao', max_iter = 100,
                     num_dissim=euclidean_dissim, cat_dissim=matching_dissim, 
                     n_init =10, gamma=None, verbose = 1, random_state=77, n_jobs=1) 
clusters = kproto.fit_predict(X_norm, categorical=[1,2,3])

Init: initializing centroids
Init: initializing clusters
Init: initializing centroids
Init: initializing clusters
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run: 1, iteration: 1/100, moves: 166, ncost: 342.1306479225952
Run: 1, iteration: 2/100, moves: 72, ncost: 330.42299547391366
Run: 1, iteration: 3/100, moves: 19, ncost: 329.33221755664516
Run: 1, iteration: 4/100, moves: 35, ncost: 327.79092166534497
Run: 1, iteration: 5/100, moves: 23, ncost: 324.862051951897
Run: 1, iteration: 6/100, moves: 19, ncost: 320.0842270025509
Run: 1, iteration: 7/100, moves: 12, ncost: 317.15402823032963
Run: 1, iteration: 8/100, moves: 16, ncost: 306.05586634183123
Run: 1, iteration: 9/100, moves: 12, ncost: 297.6660714668771
Run: 1, iteration: 10/100, moves: 5, ncost: 297.6083304454405
Run: 1, iteration: 11/100, moves: 0, ncost: 297.6083304454405
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run: 2, iteration: 1/100, moves: 195, n

### Attributes

cluster_centroids_ : array, [n_clusters, n_features]
    Categories of cluster centroids
labels_ :
    Labels of each point
cost_ : float
    Clustering cost, defined as the sum distance of all points to
    their respective cluster centroids.
n_iter_ : int
    The number of iterations the algorithm ran for.
epoch_costs_ :
    The cost of the algorithm at each epoch from start to completion.
gamma : float
    The (potentially calculated) weighing factor.

In [23]:
kproto.n_iter_

6

In [23]:
centroid = kproto.cluster_centroids_
cost = kproto.epoch_costs_ 

In [36]:
cost

[2.6402427359642435,
 2.5206138628953862,
 2.470343416047847,
 2.4574324393541227,
 2.4521737110565405,
 2.451119592103343,
 2.4511194537402847,
 2.4511194537402847]

In [24]:
def print_centroid(centroid):
    for num, cat in zip(centroid[0], centroid[1]):
        print (f"{num}{cat}")

In [25]:
print_centroid(centroid)

[1.5138275]['Hilton Garden Inn' 'S' 'Urban']
[1.35591166]['Homewood Suites' 'W' 'Suburban']
[1.02446075]['Tru by Hilton' 'S' 'Suburban']
[9.72559406]['Holiday' 'S' 'Campus']
[1.17774595]['Hilton Garden Inn' 'M' 'Suburban']
[1.17549535]['Hilton Garden Inn' 'S' 'Interstate']
[0.99309822]['Others' 'S' 'Airport']
[2.42283044]['Embassy Suites' 'M' 'Urban']
[4.56149792]['Crowne Plaza Hotels and Resorts' 'M' 'Urban']
[1.33026945]['Hilton Garden Inn' 'N' 'Urban']
[3.30688203]['Crowne Plaza Hotels and Resorts' 'S' 'Suburban']
[2.65278651]['Hilton Garden Inn' 'N' 'Urban']
[2.26685463]['Doubletree by Hilton' 'S' 'Resort']


In [52]:
kproto.n_iter_

5

In [53]:
kproto.gamma

0.2

### prepare prediction datasets

In [26]:
df = data.df.copy()
df['cluster'] = clusters
X_new = df["cluster"].to_numpy()
y_new = data.target_spor().to_numpy()

In [27]:
test_cluster_prediction(X_new, y_new)

Train Data MSE: 0.57
Train Data R2: 0.03
Holdout Data MSE: 0.42
Holdout Data R2: 0.01


In [28]:
#(503- x_min) / (x_max - x_min)
#503/norm_X
#(x-mu)/sigma

x = [(503/norm_X)*40, 'Crowne Plaza Hotels and Resorts', 'Airport', utils.region('IL')]
y_hat = predict_one(kproto, x)

In [29]:
y_hat

8

### List of Cluster

In [30]:
#number of properties for each cluster
df.groupby(by="cluster")['property_code'].count() 

cluster
0     59
1     40
2     96
3      2
4     44
5     77
6     27
7     19
8     16
9     41
10    14
11    19
12    12
Name: property_code, dtype: int64

In [31]:
from tabulate import tabulate
lst_columns = ['property_name', 'flag_name', 'num_of_rooms', 'city', 'state', 'location_type', 'spor', 'profit_margin']
for i in range(df["cluster"].max()+1):
    print(tabulate(df[df['cluster'] == i][lst_columns].head(5), headers='keys', tablefmt='psql'))
    #df[df['cluster'] == i][lst_columns].head(5)

+----+------------------------------------------+---------------------------------+----------------+---------------+---------+-----------------+----------+-----------------+
|    | property_name                            | flag_name                       |   num_of_rooms | city          | state   | location_type   |     spor |   profit_margin |
|----+------------------------------------------+---------------------------------+----------------+---------------+---------+-----------------+----------+-----------------|
|  2 | Avid Hotel Oklahoma City - Quail Springs | Avid                            |             87 | Oklahoma City | OK      | Suburban        | 1.21898  |        0.539813 |
| 15 | Crowne Plaza Baltimore                   | Crowne Plaza Hotels and Resorts |            385 | Baltimore     | MD      | Urban           | 0.466343 |        0.668021 |
| 18 | Crowne Plaza Columbus-Downtown           | Crowne Plaza Hotels and Resorts |            419 | Columbus      | OH      | Urb

In [32]:
pd.options.display.max_rows = 100

In [35]:
df[df['cluster'] == 1][lst_columns].sort_values(by='flag_name')

,property_name,flag_name,num_of_rooms,city,state,location_type,spor,profit_margin
1,Avid Hotel Meridian,Avid,87,Oklahoma City,OK,Airport,0.564489,0.510323
16,Crowne Plaza Charleston Airport,Crowne Plaza Hotels and Resorts,166,North Charleston,SC,Suburban,1.179861,0.644024
24,Crowne Plaza Lansing West,Crowne Plaza Hotels and Resorts,212,Lansing,MI,Suburban,1.209513,0.596965
25,Crowne Plaza Miami Airport,Crowne Plaza Hotels and Resorts,190,Miami,FL,Urban,3.500201,0.615067
29,Crowne Plaza Orlando Downtown,Crowne Plaza Hotels and Resorts,227,Orlando,FL,Suburban,2.995200,0.693418
378,Hotel MdR Marina Del Ray - a DoubleTree by Hilton,Doubletree by Hilton,283,Marina del Ray,CA,Interstate,0.393125,0.605302
52,Embassy Suites Bloomington,Embassy Suites,234,Bloomington,MN,Suburban,1.412074,0.619983
55,Embassy Suites Crystal City National Airport,Embassy Suites,267,Arlington,VA,Airport,2.538994,0.710580
66,Embassy Suites by Hilton Laredo,Embassy Suites,320,Laredo,TX,Urban,1.165768,0.612695
70,Embassy Suites by Hilton Washington DC Convent...,Embassy Suites,384,Washington,DC,Urban,2.940062,0.719040


In [34]:
df.groupby(by="flag_name")["property_code"].count()

flag_name
Aston                                1
Avid                                 3
Best Western Plus                    1
Candlewood Suites                    5
Comfort Suites                       2
Crowne Plaza Hotels and Resorts     25
Curio by Hilton                      3
Doubletree by Hilton                15
Embassy Suites                      18
Hampton Inn                         42
Hampton Inn and Suites              36
Hilton                               6
Hilton Garden Inn                  152
Hilton Suites                        1
Holiday Inn                          1
Holiday Inn Express                  4
Holiday Inn Express & Suites         3
Home2                               23
Homewood Suites                     38
Hotel Indigo                         1
Independent                          1
Intercontinental Hotels Group        1
La Quinta Inns and Suites            1
Liv Dev                              2
Quality Suites                       1
Red Lion Hotel 

In [ ]:
# Save models if are satisfied with clusters 

In [24]:
import pickle

with open('../web/data/kproto.pkl', 'wb') as handle:
    pickle.dump(kproto, handle, protocol=pickle.HIGHEST_PROTOCOL)
df.to_pickle('../web/data/model.pkl')